# Импорт

In [27]:
!pip install pymorphy2

In [28]:
# при работе через Google Colab
# прописать путь до всех файлов py
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/MyDrive/Учёба/8 семестр/диплом/Проект/VKR/Блокноты')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
import AnalysisFunc
import BaseFuncs

In [30]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from datetime import datetime
from math import sqrt

In [31]:
directory_name = "Учет контрольных"

# Конвертация

Колонки:



In [ ]:
# открыть через виндовс
file_name = BaseFuncs.OpenFileNameWindows()
xls_array = BaseFuncs.GetDataFrameListFromFileRaw(file_name)

In [72]:
# открыть через колаб
file_name = "/content/drive/My Drive/Учёба/8 семестр/диплом/Проект/VKR/data/Данные_ПиОА_2023_1семестр.xlsx"
# file_name = "/content/drive/My Drive/Учёба/8 семестр/диплом/Проект/VKR/data/трек HARD.xlsx"
xls_array = BaseFuncs.GetDataFrameListFromFileRaw(file_name)

In [71]:
# конвертирование данных от ИОТ
def GetDataTest(data, ind_pres, ind_score):
    score = -1
    if ind_score != -1:
        score = data.loc[ind_score, "Оценка за предметы контроля"]
        if not pd.isna(score):
            score = (float(score) if "." in str(score) else float(str(score).replace(",", ".")))
    presence = 0
    if (data.loc[ind_pres, "Оценка за предметы контроля"]=="П"):
        presence = 1
    return presence, score

temp_row = ""
for sheet in xls_array:
    for name, group in sheet.groupby(["Название РМУП"]):
        if "Предмет контроля" not in group.columns:
            continue
        name_column_test = ""
        if "Контрольная работа" in set(list(group["Предмет контроля"])):
            name_column_test = "Контрольная работа"
        else:
            name_column_test = "Работа на учебной встрече"
        results = pd.DataFrame(columns=["ФИО", "Команда", "Пол", "Баллы", "Не сдал(-а)"])

        name_discipline = group.head(1)["Название РМУП"].values[0]
        # try:
        if (group["Название встречи"].str.contains('Контрольная ').sum()==0 and name_column_test!="Контрольная работа"):
            continue
        print(name_discipline, end=":")
        print(name_column_test, set(list(group["Предмет контроля"])))

        # max_scores_before_test = {}

        percent = 0
        count_presence = summ_presence = summ_goals_btest = count_goals = count_meetings = 0
        goals_list = []

        startTime = datetime.now()

        skip = ind_presence = ind_score = 0
        # номер контрольной
        test_number = 1

        group.reset_index(inplace = True)

        for index, row in group.iterrows ():

            if (index%(len(group)//10)==0):
                print(f"{percent}%. Записей в итоге: {len(results)}")
                percent+=10

            if skip > 0:
                skip-=1
                continue

            if temp_row == "" or temp_row != row["ФИО студента"]:
                test_number = 1
                count_presence = summ_presence = summ_goals_btest = count_goals = count_meetings = 0

                temp_row = row["ФИО студента"]

                results.loc[-1] = [temp_row, ", ".join(set(group.loc[(group["ФИО студента"] == temp_row), "Команда"])), BaseFuncs.GetGender(temp_row), row["Итог ТУ"], -1] + [""]*(len(results.columns)-5)
                results.index = results.index + 1  # shifting index

            if ("Контрольная " in row["Название встречи"] or "Контрольная работа" == row["Предмет контроля"]):
                meeting_name = f"Контрольная работа {test_number}"

                if (f"Контрольная работа {test_number}" not in list(results.columns)):
                    if (f"Посещение до {meeting_name}" not in list(results.columns)):
                        results[f"Посещение до {meeting_name}"] = 0.0
                        results[f"Баллы до {meeting_name}"] = 0.0
                        results[f"Количество баллов до {meeting_name}"] = 0.0
                        results[f"Корень дисперсии баллов до {meeting_name}"] = 0.0
                        results[f"Доставленные баллы на {meeting_name}"] = -1
                        # results[f"Процент баллов до {meeting_name}"] = 0
                        results[meeting_name] = 0
                        results[f"Присутствие на {meeting_name}"] = 0.0

                if (row["Предмет контроля"] == name_column_test):

                    test_score = float(row["Оценка за предметы контроля"]) if row["Оценка за предметы контроля"]!="" else 0
                    results.loc[((results['ФИО'] == temp_row)), f"Контрольная работа {test_number}"] = test_score

                    # определение посещамесоти КР и дополнительных баллов
                    if "Контрольная работа" == row["Предмет контроля"]:
                        if  group.loc[index + 1, "Название встречи"] != row["Название встречи"]:
                            if group.loc[index - 1, "Предмет контроля"] == "Посещение":
                                ind_presence = index - 1
                                ind_score = index - 2
                            else:
                                ind_presence = index - 2
                                ind_score = index - 1
                        elif group.loc[index + 2, "Название встречи"] != row["Название встречи"]:
                            skip = 1
                            if group.loc[index + 1, "Предмет контроля"] == "Посещение":
                                ind_presence = index + 1
                                ind_score = index - 1
                            else:
                                ind_presence = index - 1
                                ind_score = index + 1
                        else:
                            skip = 2
                            if group.loc[index + 2, "Предмет контроля"] == "Посещение":
                                ind_presence = index + 2
                                ind_score = index + 1
                            else:
                                ind_presence = index + 1
                                ind_score = index + 2
                    else:
                        if group.loc[index + 1, "Название встречи"] != row["Название встречи"]:
                            ind_presence, ind_score = index - 1, -1
                        else:
                            skip = 1
                            ind_presence, ind_score = index + 1, -1

                    test_presence, addit_score = GetDataTest(group, ind_presence, ind_score)
                    if ind_presence < index:
                        count_presence -= 1
                        summ_presence -= test_presence
                    if ind_score < index:
                        if addit_score>0:
                            summ_goals_btest -= addit_score
                            results.loc[((results['ФИО'] == temp_row)), "Баллы"] -= addit_score
                            count_goals -= 1
                            goals_list.pop()


                    if count_presence == 0:
                        continue
                    results.loc[((results['ФИО'] == temp_row)), f"Посещение до {meeting_name}"] = round(summ_presence / float(count_presence), 2)
                    results.loc[((results['ФИО'] == temp_row)), f"Баллы до {meeting_name}"] = summ_goals_btest
                    results.loc[((results['ФИО'] == temp_row)), f"Количество баллов до {meeting_name}"] = count_goals
                    results.loc[((results['ФИО'] == temp_row)), f"Корень дисперсии баллов до {meeting_name}"] = sqrt(np.var(goals_list))
                    results.loc[((results['ФИО'] == temp_row)), f"Доставленные баллы на {meeting_name}"] = addit_score
                    results.loc[((results['ФИО'] == temp_row)), f"Присутствие на {meeting_name}"] = test_presence

                    count_presence = summ_presence = summ_goals_btest = count_goals = count_meetings = 0
                    goals_list = []
                    test_number+=1


            elif (row["Предмет контроля"] != "Посещение"):
                count_meetings += 1
                if not pd.isna(row["Оценка за предметы контроля"]):
                    goal = float(row["Оценка за предметы контроля"])
                    goals_list.append(goal)
                    summ_goals_btest += goal
                    if goal>0.0:
                        count_goals += 1
                else:
                    goals_list.append(0)
            else:
                count_presence += 1
                if row["Оценка за предметы контроля"]=="П":
                    summ_presence += 1


        results = results.fillna(0)
        results = results.replace("", 0)
        results = results.loc[:, (results != 0).any(axis=0)]
        results = results[results.columns[:len(results.columns)-(len(results.columns)-5)%7]]
        results = results.replace(-1, 0)
        results.loc[results["Баллы"]<61.0, 'Не сдал(-а)'] = 1
        # results = results.drop(results.columns[-7:], axis= 1)

        print(f"Время выполнения: {datetime.now()-startTime}")
        print("Отдельное сохранение...")

        BaseFuncs.SaveFileCSV(path = f"/content/drive/My Drive/Учёба/8 семестр/диплом/Проект/VKR/{directory_name}", filename = f"{name_discipline}", results = results)
        print("Сохранено")
        display(results.loc[:,:"Присутствие на Контрольная работа 2"].head(5))
        # except Exception as e:
        #     print(e)



Алгебра:Работа на учебной встрече {'Посещение', 'Работа на учебной встрече'}
0%. Записей в итоге: 0
10%. Записей в итоге: 96
20%. Записей в итоге: 191
30%. Записей в итоге: 286
40%. Записей в итоге: 381
50%. Записей в итоге: 477
60%. Записей в итоге: 572
70%. Записей в итоге: 668
80%. Записей в итоге: 762
90%. Записей в итоге: 857
100%. Записей в итоге: 952
Время выполнения: 0:00:25.264996
Отдельное сохранение...
Сохранено


,ФИО,Команда,Пол,Баллы,Не сдал(-а),Посещение до Контрольная работа 1,Баллы до Контрольная работа 1,Количество баллов до Контрольная работа 1,Корень дисперсии баллов до Контрольная работа 1,Доставленные баллы на Контрольная работа 1,Контрольная работа 1,Присутствие на Контрольная работа 1,Посещение до Контрольная работа 2,Баллы до Контрольная работа 2,Количество баллов до Контрольная работа 2,Корень дисперсии баллов до Контрольная работа 2,Доставленные баллы на Контрольная работа 2,Контрольная работа 2,Присутствие на Контрольная работа 2
951,Абатуров Георгий Сергеевич,"АЛГЕБРА Л-08, АЛГЕБРА П-08.02",1,76.0,0,0.55,0.5,1.0,0.143740,0,16.0,1.0,1.0,0.0,0.0,0.000000,0,11.5,1.0
950,Абдимиталипов Мустафа,"АЛГЕБРА П-04.01, АЛГЕБРА Л-04",1,61.0,0,0.82,0.0,0.0,4.281947,0,12.0,1.0,1.0,0.0,0.0,0.000000,0,12.0,1.0
949,Абдимуминов Абдужалол Абдихаким угли,"АЛГЕБРА Л-05, АЛГЕБРА П-05.04",1,0.0,1,0.36,0.0,0.0,6.642781,0,0.0,0.0,0.0,0.0,0.0,0.000000,0,0.0,0.0
948,Абдразакова Екатерина Наилевна,"АЛГЕБРА Л-04, АЛГЕБРА П-04.02",0,100.0,0,0.82,8.0,4.0,0.955814,0,20.0,1.0,0.6,9.0,3.0,1.469694,0,20.0,1.0
947,Абдрашитова Диана Дамировна,"АЛГЕБРА Л-07, АЛГЕБРА П-07.02",0,61.0,0,0.73,1.0,1.0,3.658661,0,17.0,1.0,0.8,0.0,0.0,0.000000,0,15.0,1.0


Программирование и основы алгоритмизации 1:Контрольная работа {'Посещение', 'Контрольная работа', 'Работа на учебной встрече'}
0%. Записей в итоге: 0
10%. Записей в итоге: 97
20%. Записей в итоге: 193
30%. Записей в итоге: 289
40%. Записей в итоге: 385
50%. Записей в итоге: 481
60%. Записей в итоге: 578
70%. Записей в итоге: 674
80%. Записей в итоге: 769
90%. Записей в итоге: 865
100%. Записей в итоге: 961
Время выполнения: 0:00:25.031744
Отдельное сохранение...
Сохранено


,ФИО,Команда,Пол,Баллы,Не сдал(-а),Посещение до Контрольная работа 1,Баллы до Контрольная работа 1,Количество баллов до Контрольная работа 1,Корень дисперсии баллов до Контрольная работа 1,Доставленные баллы на Контрольная работа 1,Контрольная работа 1,Присутствие на Контрольная работа 1,Посещение до Контрольная работа 2,Баллы до Контрольная работа 2,Количество баллов до Контрольная работа 2,Корень дисперсии баллов до Контрольная работа 2,Доставленные баллы на Контрольная работа 2,Контрольная работа 2,Присутствие на Контрольная работа 2
960,Абатуров Георгий Сергеевич,"ПиОА П-08.02, ПиОА Л-08",1,76.00,0,0.93,16.0,8.0,0.997775,0.0,12.00,1.0,1.00,6.0,3.0,0.000000,0.0,12.00,1.0
959,Абдимиталипов Мустафа,"ПиОА П-04.03, ПиОА Л-04",1,24.07,1,1.00,3.5,3.0,1.267166,0.0,7.38,1.0,1.00,1.2,2.0,0.294392,0.0,4.49,1.0
958,Абдимуминов Абдужалол Абдихаким угли,"ПиОА П-05.04, ПиОА Л-05",1,0.00,1,0.00,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0,0.0,0.000000,0.0,0.00,0.0
957,Абдразакова Екатерина Наилевна,"ПиОА П-04.01 , ПиОА Л-04",0,87.67,0,1.00,15.0,8.0,0.962983,0.0,11.70,1.0,1.00,6.0,3.0,0.000000,2.0,11.80,1.0
956,Абдрашитова Диана Дамировна,"ПиОА П-07.02, ПиОА Л-07",0,63.56,0,0.86,16.0,8.0,1.930850,0.0,4.48,1.0,0.33,6.0,3.0,0.000000,0.0,10.08,1.0


Математический анализ:Контрольная работа {'Практическое задание', 'Письменный ответ', 'Работа на учебной встрече', 'Посещение', 'Контрольная работа'}
0%. Записей в итоге: 0
10%. Записей в итоге: 96
20%. Записей в итоге: 191
30%. Записей в итоге: 287
40%. Записей в итоге: 382
50%. Записей в итоге: 477
60%. Записей в итоге: 573
70%. Записей в итоге: 668
80%. Записей в итоге: 763
90%. Записей в итоге: 858
100%. Записей в итоге: 953
Время выполнения: 0:00:21.717183
Отдельное сохранение...
Сохранено


,ФИО,Команда,Пол,Баллы,Не сдал(-а),Посещение до Контрольная работа 1,Баллы до Контрольная работа 1,Количество баллов до Контрольная работа 1,Корень дисперсии баллов до Контрольная работа 1,Доставленные баллы на Контрольная работа 1,Контрольная работа 1,Присутствие на Контрольная работа 1,Посещение до Контрольная работа 2,Баллы до Контрольная работа 2,Количество баллов до Контрольная работа 2,Корень дисперсии баллов до Контрольная работа 2,Доставленные баллы на Контрольная работа 2,Контрольная работа 2,Присутствие на Контрольная работа 2
952,Абатуров Георгий Сергеевич,"МА П-08.02, МА Л-08",1,76.0,0,1.00,0.0,0.0,0.000000,0.0,15.0,1.0,1.0,20.0,1.0,5.527708,0.0,38.0,1.0
951,Абдимиталипов Мустафа,"МА Л-04, МА П-04.01",1,46.0,1,0.86,2.0,1.0,0.845484,0.0,1.0,1.0,0.7,23.0,3.0,4.231988,0.0,20.0,1.0
950,Абдимуминов Абдужалол Абдихаким угли,"МА Л-05, МА П-05.04",1,0.0,1,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
949,Абдразакова Екатерина Наилевна,"МА Л-04, МА П-04.02",0,100.0,0,0.79,24.0,8.0,1.704773,0.0,29.0,1.0,1.0,22.0,5.0,5.189466,0.0,25.0,1.0
948,Абдрашитова Диана Дамировна,"МА П-07.02, МА Л-07",0,64.0,0,0.93,0.0,0.0,0.000000,0.0,9.0,1.0,0.5,20.0,1.0,5.527708,0.0,35.0,1.0


In [73]:
# очистка названий колонок и замена на первую строку в данны от Modeus
for sheet in xls_array:
    sheet.columns = sheet.loc[0, :].values.flatten().tolist()
    sheet.drop(index=[0], inplace=True)
    sheet = sheet.reset_index(drop=True)

In [74]:
results = pd.DataFrame(columns=["ФИО", "Команда", "Направление", "Пол", "Баллы", "Не сдал(-а)"])
print(results.columns)

for sheet in xls_array:
    print(f"Записей: {len(sheet)}")
    startTime = datetime.now()
    percent = 0

    # try:
    # max_scores_before_test = {}
    count_presence = summ_presence = count_kontr = 0
    summ_goals_btest = count_goals = count_meetings = summ_goals_result = 0

    for index, row in sheet.iterrows ():
        if (index%(len(sheet)//10)==0):
            print(f"{percent}%. Записей в итоге: {len(results)}")
            percent+=10
        results.loc[-1] = [row["Обучающийся"], row["Учебные команды"], row["Направление подготовки"], BaseFuncs.GetGender(row["Обучающийся"]), row["Итог текущ."], 0] + [""]*(len(results.columns)-6)
        results.index = results.index + 1

        count_presence = summ_presence = count_kontr = 0
        summ_goals_btest = count_goals = count_meetings = summ_goals_result = 0

        goals_list = []
        for i in range(5, len(sheet.columns.to_list())):
            if sheet.columns.to_list()[i]!="Контрольная работа":
                if sheet.columns[i]=="Присутствие":
                    count_presence+=1
                    if (row[i]=="П"):
                        summ_presence+=1
                elif sheet.columns[i]=="Работа на учебной встрече":
                    count_meetings += 1
                    if not pd.isna(row[i]):
                        goal = (float(row[i]) if "." in str(row[i]) else float(str(row[i]).replace(",", ".")))
                        goals_list.append(goal)
                        summ_goals_btest += goal
                        if goal>0.0:
                            count_goals += 1
                    else:
                        goals_list.append(0)
            else:
                count_kontr+=1
                meeting_name = f"Контрольная работа {count_kontr}"

                if (f"Посещение до {meeting_name}" not in list(results.columns)):
                    results[f"Посещение до {meeting_name}"] = 0
                    results[f"Баллы до {meeting_name}"] = 0
                    results[f"Количество баллов до {meeting_name}"] = 0.0
                    results[f"Корень дисперсии баллов до {meeting_name}"] = 0.0
                    results[f"Доставленные баллы на {meeting_name}"] = 0.0
                    results[meeting_name] = 0
                    results[f"Присутствие на {meeting_name}"] = 0.0


                count_presence -= 1
                if (row[i-1]=="П"):
                    summ_presence-=1
                    results.loc[results.index[-1], f"Присутствие на {meeting_name}"] = 1
                else:
                    results.loc[results.index[-1], f"Присутствие на {meeting_name}"] = 0

                results.loc[results.index[-1], f"Посещение до {meeting_name}"] = round(summ_presence / float(count_presence), 2)
                results.loc[results.index[-1], f"Баллы до {meeting_name}"] = summ_goals_btest
                results.loc[results.index[-1], f"Количество баллов до {meeting_name}"] = count_goals
                results.loc[results.index[-1], f"Корень дисперсии баллов до {meeting_name}"] = sqrt(np.var(goals_list))



                test_score = float(row[i]) if row[i]!="" else 0
                results.loc[results.index[-1], meeting_name]  = test_score
                if sheet.columns[i+1]=="Работа на учебной встрече":
                    if not pd.isna(row[i+1]):
                        results.loc[results.index[-1],f"Доставленные баллы на {meeting_name}"] = (float(row[i+1]) if "." in str(row[i]) else float(str(row[i+1]).replace(",", ".")))
                        i+=1
                        results.loc[results.index[-1], "Баллы"] -= results.loc[results.index[-1],f"Доставленные баллы на {meeting_name}"]
                count_presence = summ_presence = summ_goals_btest = count_goals = 0

    results = results.fillna(0)
    results = results.replace("", 0)

    results.loc[results["Баллы"]<61.0, 'Не сдал(-а)'] = 1

    print(f"Время выполнения: {datetime.now()-startTime}")
    print("Отдельное сохранение...")
    BaseFuncs.SaveFileCSV(path = f"/content/drive/My Drive/Учёба/8 семестр/диплом/Проект/VKR/{directory_name}", filename = "Программирование и основы алгоритмизации тест", results = results)
    print("Сохранено")
    display(results.loc[:,:"Присутствие на Контрольная работа 2"].head(5))
    # except Exception as e:
    #     print("Ошибка:", e)


Index(['ФИО', 'Команда', 'Направление', 'Пол', 'Баллы', 'Не сдал(-а)'], dtype='object')
Записей: 961
0%. Записей в итоге: 95
10%. Записей в итоге: 191
20%. Записей в итоге: 287
30%. Записей в итоге: 383
40%. Записей в итоге: 479
50%. Записей в итоге: 575
60%. Записей в итоге: 671
70%. Записей в итоге: 767
80%. Записей в итоге: 863
90%. Записей в итоге: 959
Время выполнения: 0:00:07.811574
Отдельное сохранение...
Сохранено


,ФИО,Команда,Направление,Пол,Баллы,Не сдал(-а),Посещение до Контрольная работа 1,Баллы до Контрольная работа 1,Количество баллов до Контрольная работа 1,Корень дисперсии баллов до Контрольная работа 1,Доставленные баллы на Контрольная работа 1,Контрольная работа 1,Присутствие на Контрольная работа 1,Посещение до Контрольная работа 2,Баллы до Контрольная работа 2,Количество баллов до Контрольная работа 2,Корень дисперсии баллов до Контрольная работа 2,Доставленные баллы на Контрольная работа 2,Контрольная работа 2,Присутствие на Контрольная работа 2
960,Абдалла Ахмед Махмуд Эльсайед Ахмед,"ПиОА АТ-04, ПиОА К-04, ПиОА Л-01, ПиОА П-04",02.03.03 Математическое обеспечение и админист...,1,22.60,1,0.22,4.8,3.0,0.733212,0.0,1.0,1.0,0.25,2.5,2.0,0.701300,0.0,0.00,0.0
959,Абдуахатова Сабрина Бехзодовна,"ПиОА АТ-30, ПиОА К-30, ПиОА Л-05, ПиОА П-30",40.03.01 Юриспруденция,0,69.14,0,0.67,16.0,8.0,0.800000,3.0,9.1,1.0,0.75,9.0,4.0,0.869227,4.0,13.04,1.0
958,Абдукаримов Толибжон Тохиржон угли,"ПиОА АТ-04, ПиОА К-04, ПиОА Л-01, ПиОА П-04",10.05.01 Компьютерная безопасность,1,3.00,1,1.00,0.0,0.0,0.000000,0.0,3.0,1.0,0.75,0.0,0.0,0.000000,0.0,0.00,1.0
957,Абдулхамидов Бекзод Шерзод угли,"ПиОА АТ-02, ПиОА К-02, ПиОА Л-01, ПиОА П-02",10.03.01 Информационная безопасность,1,17.80,1,0.89,2.0,2.0,0.458258,5.0,1.9,1.0,1.00,8.5,3.0,1.326650,0.0,1.70,1.0
956,Абдураматов Мирзамахмуд Мирзарахмат угли,"ПиОА АТ-02, ПиОА К-02, ПиОА Л-01, ПиОА П-02",10.05.01 Компьютерная безопасность,1,5.80,1,0.44,2.0,1.0,0.600000,0.0,3.8,1.0,0.25,0.0,0.0,0.498888,0.0,0.00,0.0
